### 实验目的：
- 了解网络爬虫的结构和规则；
- 理解robots.txt文件的作用和形式；
- 掌握requests模块的主要方法以及Response对象的常见属性；
- 了解BeautifulSoup模块以及如何过滤选取信息；
- 了解re模块的基本使用。

### 实验要求:
1. 运行并分析给出的示例代码；
2. 编写代码完成实例1：爬取京东商品页面；
3. 编写代码完成实例2：爬取网络图片并进行存储；
4. 编写代码完成实例3：爬取中国大学排名；
5. 编写代码完成实例4：爬取淘宝商品名称和价格。

### 网络信息的爬取

 [requests库](http://www.python-requests.org) 是用Python语言编写，基于 urllib，采用 Apache2 Licensed 开源协议的 HTTP 库。它比 urllib 更加方便，可以节约我们大量的工作，完全满足 HTTP 测试需求。
 
 通过命令 pip install requests 进行安装

In [ ]:
import requests

# 尝试获取某个网页，得到一个名为r的Response对象
r = requests.get("http://www.baidu.com")
print(r.status_code) # 状态码200，说明请求成功

In [2]:
print(type(r)) # 打印Response对象的类型

NameError: name 'r' is not defined

In [ ]:
print(r.headers) # 页面头部信息

### Requests库的7个主要方法

方法 |  说明 | 
---|---|
requests.request()  | 构造一个请求，支撑其他方法的基础方法，其他6个方法都是通过调用request()方法实现的
requests.get()     | 获取html网页的主要方法，对应于HTTP的get
requests.head()    | 获取html网页头信息的方法，对应于HTTP的head
requests.post()    | 向html网页提交post请求的方法，对应于HTTP的post
requests.put()     | 向html网页提交put请求的方法，对应于HTTP的put
requests.patch()   | 向html网页提交局部修改请求的方法，对应于HTTP的patch
requests.delete()   | 向html网页提交删除请求的方法，对应于HTTP的delete

#### r = requests.get(url,params=None,$**$kwargs)
- 方法作用：构造一个向服务器请求资源的Request对象，返回一个包含服务器资源的Response对象。Response对象包含爬虫返回的内容。
- url:获取页面的url链接。
- params:url中的额外参数，字典或字节流格式，可选。
- $**$kwargs:12个控制访问的参数，可选。

### Response对象的属性 (这里以r为例说明)

属性 |  说明 | 
---|---|
r.status_code  | http请求的返回状态，200表示连接成功，404表示连接失败
r.text     | http响应内容的字符串形式，即url对应的页面内容
r.encoding    | 从http header中猜测的响应内容编码方式；如果header中不存在charset，则认为编码为ISO-8859-1(这个编码没法解析中文)
r.apparent_encoding    | 从内容中分析出的响应内容编码方法（备选编码方式）
r.content     | http响应内容的二进制形式

In [ ]:
print(r.encoding) # 打印文本编码

In [ ]:
print(r.text) # 打印网页内容

In [ ]:
print(r.apparent_encoding) 

In [ ]:
r.encoding = 'utf-8' # 更改编码方式
print(r.text) # 使用新的编码方法打印网页内容

#### 爬取网页的通用代码框架

In [ ]:
# 网络连接有风险，异常处理很重要
def getHTMLText(url):
    try:
        r = requests.get(url,timeout=30) 
        # timeout参数使得requests在经过设定的秒数时间之后停止等待响应
        r.raise_for_status() #如果状态不是200，引发HTTPError异常
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return "产生异常"

In [ ]:
url = "http://www.baidu.com"
print(getHTMLText(url))

In [ ]:
url = "www.baidu.com"
print(getHTMLText(url))

网站服务器可以限制网络爬虫：
1. 来源审查：判断User-Agent进行限制；检查来访HTTP协议头的User-Agent域，只响应浏览器或友好爬虫的访问；
2. 发布公告：Robots协议，告知所有爬虫网站的爬取策略，要求爬虫遵守。

Robots协议：
- Robots Exclusion Standard,网络爬虫排除标准；
- 作用：网站告知网络爬虫哪些页面可以抓取，哪些不行；
- 形式：robots.txt放在网站根目录下，比如：
    - 豆瓣的：https://www.douban.com/robots.txt 
    - 京东的：https://www.jd.com/robots.txt  
    - 新浪新闻: https://www.news.xina.com.cn/robots.txt 
    - QQ: https://www.qq.com/robots.txt
- 使用：
    - 网络爬虫自动或人工识别robots.txt，再进行内容爬取；
    - Robots协议是建议不是约束，网络爬虫可以不遵守，但存在法律风险；
- 基本语法：
    - User-agent:*(代表所有)
    - Disallow:/（代表根目录）

#### 实例1：爬取京东商品页面

In [ ]:
url = "http://item.jd.com/11993134.html"
print(getHTMLText(url))

使用之前定义的爬取网页通用代码框架并没有输出网页，是因为网站的反爬程序。

解决方法是补充上headers参数(headers的形式为字典)，目的是模拟浏览器，欺骗服务器，从而获取和浏览器一致的内容。

如何查看浏览器的代理信息user-agent:
1. 浏览器输入：about:version
2. 访问 http://httpbin.org/get

In [62]:
# 修改爬取网页的通用代码框架
def getHTMLText2(url):
    try:
        head = {
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36",
"cookie":"fxl2cfwarlvgkwn1f250ah06wisev1ya"

        }     
        r = requests.get(url,headers=head,timeout=30) 
        # timeout参数使得requests在经过设定的秒数时间之后停止等待响应
        r.raise_for_status() #如果状态不是200，引发HTTPError异常
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return "产生异常"




In [64]:
# 京东商品页面爬取
url = "http://item.jd.com/11993134.html"
print(getHTMLText2(url)[:1000]) # 查看前1000个字符

<!DOCTYPE HTML>
<html lang="zh-CN">
<head>
    <!-- shouji -->
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
    <title>《Python编程 从入门到实践 第2版（图灵出品）》([美]埃里克·马瑟斯)【摘要 书评 试读】- 京东图书</title>
    <meta name="keywords" content="Python编程 从入门到实践 第2版（图灵出品）,,人民邮电出版社,9787115546081,,在线购买,折扣,打折"/>
    <meta name="description" content="京东JD.COM图书频道为您提供《Python编程 从入门到实践 第2版（图灵出品）》在线选购，本书作者：，出版社：人民邮电出版社。买图书，到京东。网购图书，享受最低优惠折扣!" />
    <meta name="format-detection" content="telephone=no">
    <meta http-equiv="mobile-agent" content="format=xhtml; url=//item.m.jd.com/product/11993134.html">
    <meta http-equiv="mobile-agent" content="format=html5; url=//item.m.jd.com/product/11993134.html">
    <meta http-equiv="X-UA-Compatible" content="IE=Edge">
    <link rel="canonical" href="//item.jd.com/11993134.html"/>
        <link rel="dns-prefetch" href="//misc.360buyimg.com"/>
    <link rel="dns-prefetch" href="//static.360buyimg.com"/>
    <link rel="dns-prefetch" href="//img10.360buy

####  实例2. 网络图片的爬取和存储

In [ ]:
import os

# 从国家地理：http://www.ngchina.com.cn/ 选择一个图片web页面

url = 'https://ngimages.oss-cn-beijing.aliyuncs.com/2021/10/20/2cf27464-65a7-4b3a-ba85-af78898b400f.jpg'  # 图片url 
root =   '/home/huanghj/5/' # 存放图片目录
path = root + url.split('/')[-1]   # 图片路径
# print(requests.get(url).text)
try:
    if not os.path.exists(root):
        os.mkdir(root)
        # 如果图片目录不存在，则创建
    if not os.path.exists(path):
        r = requests.get(url)
        with open(path,'wb') as f:
           
            f.write(r.content)
            f.close()
            print("file save successfully")
    else:
        print("such file existed")
            
    # 如果图片路径不存在，则爬取图片并保存
    # 使用Response对象的content属性可以得到图片
    # 如果图片存在，则打印字符串“图片已存在”
    
except:
     print("爬取失败")

### 网络信息的提取

- 在命令行通过 pip install beautifulsoup4 安装
- [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/),是解析、遍历、维护“标签树”的功能库。Beautiful Soup库([官网](https://beautifulsoup.readthedocs.io/zh_CN/latest/))也叫beautifulsoup4 或 bs4，主要使用BeautifulSoup类。BeautifulSoup对应一个HTML/XML文档的全部内容；

- 标签成对出现，属性0个或多个；标签的形式 $<$标签名 标签属性$>$标签非属性字符串/注释$<$/标签名$>$。

例如：$<$p class="title"$><$b$>$The dormouse's story$<$/b$>$$<$/p$>$

In [ ]:
from bs4 import BeautifulSoup

html = requests.get('https://python123.io/ws/demo.html')
print(html.text)

soup = BeautifulSoup(html.text, 'html.parser')
print(soup.title)
print(type(soup))

Beautiful Soup库解析器

解析器 |  使用方法 |  条件 
---|---|---|
bs4的HTML解析器  | BeautifulSoup(mk,'html.parser')  | 安装bs4库
lxml的HTML解析器  | BeautifulSoup(mk,'lxml')      | pip install lxml
lxml的XML解析器  | BeautifulSoup(mk,'xml')       | pip install lxml
html5lib的解析器  | BeautifulSoup(mk,'html5lib')    | pip install html5lib

BeautifulSoup类的基本元素

基本元素 | 说明 
---|---|
Tag  | 标签，最基本的信息组织单元，分别用<>和</>表明开头和结尾
Name  | 标签的名字，如标签$<$p $>$...$<$/p$>$的名字是p,格式$<$tag$>$.name
Attributes  |  标签的属性，如标签$<$p class="title"$>$...$<$/p$>$的属性class, 格式：$<$tag$>$.attrs，输出以字典形式组织
NavigablesString  | 标签内非属性字符串，如标签$<$$>$...$<$/$>$中...代表的字符串，格式：$<$tag$>$.string；可以跨越多个层次
Comment | 标签内字符串的注释部分，一种特殊的Comment类型

In [ ]:
# 任何存在于HTML语法中的标签都可以用soup.<tag>访问获得
# 当HTML文档中存在多个相同<tag>对应内容时，soup.<tag>返回第一个
print(soup.a) 
print(soup.a.name)  # 标签的名字
print(soup.a.parent.name) # 父亲标签的名字
print(soup.a.parent.parent.name) 

In [ ]:
tag = soup.a
print(type(tag))

print(tag.attrs)
print(type(tag.attrs)) #标签的属性

print(tag['class'])
print(tag.string) # 非属性字符串

In [ ]:
print(soup.p)
print(soup.p.string)

In [ ]:
print(type(soup.p.string))

In [ ]:
newsoup = BeautifulSoup("<b><!--This is a comment--></b><p>This is not a comment</p>","html.parser")

print(newsoup.b.string)
print(type(newsoup.b.string))

print(newsoup.p.string)
print(type(newsoup.p.string))

In [ ]:
import requests
from bs4 import BeautifulSoup

html = requests.get('https://python123.io/ws/demo.html')
soup = BeautifulSoup(html.text, 'html.parser')

# 提取HTML中所有URL链接
for link in soup.find_all('a'):
    print(link.get('href'))

####  soup.find_all(name,attrs,recursive,string,**kwargs)
- 返回一个列表类型，存储查找的结果
-  name: 对标签名称的检索字符串
- attrs: 对标签属性值得检索字符串，可标注属性检索
- recursive: 是否对子孙全部检索，默认True
- string: 标签中字符串区域的检索字符串

In [ ]:
for tag in soup.find_all(True):
    print(tag.name)

如果一个html文件中出现了标签名相同，属性不同的标签，可以用

- soup.find(tagname,attrs={attrName:attrValue}) 
- soup.find(tagname,attrValue)

获取Tag的位置。

find()和find_all()类似，只不过find返回的是第一个匹配结果，find_all返回的是所有匹配结果

In [ ]:
content = soup.find_all('p','course')
print(len(content))
print(content)

In [ ]:
print(soup.find_all('a',recursive=True))

In [ ]:
print(soup.find_all('a',recursive=False))

In [ ]:
print(soup.find_all(string="Basic Python"))

#### 实例3：中国大学排名定向爬虫实例

功能描述
- 输入：大学排名URL链接
- 输出：大学排名信息（排名，大学名称，总分），结果保存到excel文件
- 技术路线：bs4,requests
- 定向爬虫：仅对输入URL进行爬取，不扩展爬取

程序的结构设计：
1. 从网络上获取大学排名网页内容，编写函数 getHTMLText()
2. 提取网页内容中信息到合适的数据结构，编写函数 fillUnivList()
3. 将结果存储到excel文件中，编写函数 saveUnivList()

In [ ]:
# 最好大学网 http://www.zuihaodaxue.cn/ 
# 该网址已经失效，请自行查找大学排名网站并爬取相关信息
# 没有robots.txt文件
import requests
from bs4 import BeautifulSoup
import bs4
import openpyxl

# 爬取网页框架
def getHTMLText(url):
    '''函数带一个参数url，爬取网页内容并返回'''
    r = requests.get(url)
    r.encoding = r.apparent_encoding
    return r
    pass

In [ ]:
def fillUnivList(ulist, html):
    '''函数带两个参数，参数ulist是一个二维列表，参数html是一个字符串，
    函数解析网页内容html，将网页中的大学排名信息（排名，大学名称，总分）保存到ulist中
    '''
    soup = BeautifulSoup(html, 'html.parser')
 
    for row in soup.find_all('tr'):
        tmp = []
        for rank in row.find_all('td')[0:1]:
            tmp.append(rank.string.strip().replace('\n',''))
            
        for name in row.find_all('a','name-cn'):
            tmp.append(name.string)
        for score in row.find_all('td')[4:5]:
            tmp.append(score.string.strip().replace('\n',''))
    
        ulist.append(tmp)
#         print(ulist)
    return ulist
    pass

    

In [ ]:
def saveUnivList(ulist,filename):
    '''函数带有两个参数，猜数ulist是一个二维列表，每一项是一个列表，保存了（排名，大学名称，总分）
    参数filename是要保存的文件名；
    函数将列表内容保存为一个excel文件'''
    ws = openpyxl.Workbook()
    wb = ws.active
    wb.title = '中国最好大学排名'
    wb['A1'] = '排名' 
    wb['B1'] = '大学'
    wb['C1'] = '总分'
    for row in ulist:
            wb.append(row)
    ws.save(filename)
    pass

In [ ]:
uinfo = []
url = 'https://www.shanghairanking.cn/rankings/bcur/2021'
filename = 'rank.xlsx'
html = getHTMLText(url)
fillUnivList(uinfo, html)
saveUnivList(uinfo,filename)

#### Re(正则表达式）库入门

正则表达式是字符串处理的有力工具，正则表达式使用预定义的模式去匹配一类具有共同特征的字符串，可以快速、准确地完成复杂的查找、替换等处理要求。相比字符串自身的方法，RE 提供了更强大的处理功能。

正则表达式由字符和操作符构成，通过编译将符合正则表达式语法的字符串转换成正则表达式特征。

正则表达式的表示类型：
  - raw string类型（原生字符串类型），re库采用raw string类型表示正则表达式，表示为r'text'；raw string是不包含对转义符再次转义的字符串；
  - re库也可以采用string类型表示正则表达式；

#### 正则表达式的常用操作符

操作符 | 说明  | 实例 
---|---|---|
.     | 匹配除换行符以外的任何单个字符   |   
[]    | 匹配字符集，对单个字符给出取值范围  | [abc]表示a、b、c；[a-z]表示a到z的单个字符
[^]   | 匹配非字符集，对单个字符给出排除范围   | [^abc]表示非a或b或c的单个字符
\d  | 匹配数字[0-9]  |
\D(^\d)  | 匹配非数字  |
\s   | 匹配任何空白字符，包括空格，制表符，换页符 | 
\S(^\s)  | 匹配非空白字符 |
\w  | 匹配单词字符 [a-zA-Z0-9]
\W  | 匹配非单词字符
{m} 或{m,n} | 匹配前一个字符 m次 或 m至n次（含n） | ab{2}c表示abbc；ab{1,2}c表示abc,abbc 
^ | 匹配字符串开头 | ^abc表示abc且在一个字符串的开头
\$ | 匹配字符串结尾 | abc\$表示abc且在一个字符串的结尾
() | 将()内的内容作为一个整体来对待  | (abc)表示abc,(abc$|$def)表示abc、def
\* | 匹配 $*$ 之前的字符或子模式0次或无限次 | abc\*表示ab,abc,abcc，abccc等
\+ | 匹配 $+$ 之前的字符或子模式1次或无限次扩展  | abc+表示abc,abcc,abccc等
? | 匹配 ？ 之前的 0 个字符或 1 个字符或子模式   | abc?表示ab,abc
$|$  | 匹配位于 $|$ 之前或之后的字符或者表达式  | abc $|$ def表示abc、def 

#### re模块中的主要功能函数

函数 | 说明 
---|---|
re.search()  | 在一个字符串中搜索匹配正则表达式的第一个位置，返回match对象或None
re.match() | 从一个字符串的开始位置起匹配正则表达式，返回match对象或None
re.findall()  | 搜索字符串，以列表类型返回全部能匹配的子串
re.split() | 将一个字符串按照正则表达式匹配结果进行分割，返回列表类型
re.finditer() | 搜索字符串，返回一个匹配结果的迭代类型，每个迭代元素是match对象
re.sub() | 在一个字符串中替换所有匹配正则表达式的子串，返回替换后的字符串
re.compile() | 创建模式对象

#### 实例4：淘宝商品比价定向爬虫

目标：获取淘宝搜索页面的信息，提取其中的商品名称和价格

功能描述：
- 输入：商品搜索URL链接
- 输出：商品价格信息（序号，价格，名称）
- 技术路线：requests，re
- 定向爬虫可行性：https://s.taobao.com/robots.txt 
    - User-agent: *
    - Disallow: /

重点：淘宝的搜索接口；翻页的处理

程序的结构设计：
1. 提交商品搜索请求，循环获取页面
2. 对于每个页面，提取商品名称和价格信息
3. 将爬取的信息保存到excel文件中

提示：目前淘宝不支持直接搜索，需要登录之后才能搜索商品。所以，先登录账号，然后搜索商品查看网页源代码，找到 Accept和Cookie信息，将其添加到headers中。

In [60]:
# 程序模块
import requests
import re
import openpyxl

def getHTMLText(url):
    """返回url页面信息的字符串形式"""
    headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",      
    "cookie":"cna=BOUbGNZnL2YCAXUeYxWbNjKo; tracknick=tb004474346; enc=BadH+UNhwlKQjyWdnuHnGF6IcF+jrez8K99ZdF4YOkCQFGCxnrcP9SZqWSMJ7Fb9+tsU6tumzLHDx9LDwBymKQ==; miid=120135379771227871; _m_h5_tk=bf9868dea029879d6959335b4a58fe07_1635532129090; _m_h5_tk_enc=87317efa3781b56506defb99d7864e04; _samesite_flag_=true; cookie2=1a0164e7e5ec639ddc287585df1a612c; t=b421bd4fc97a2ceaa78495abbea36a75; _tb_token_=fe5341431ba35; xlly_s=1; sgcookie=E100m4nTPK5tJqCtQEtSobU8faDFoNiV4CXri0FD+qQWIH7ImvEpP+7dVcG9bO/lza/BYA2ebB+4r8AapJuajGt1bmrFS2Iye9Jd+mz5krSy3EY=; unb=4122317925; uc3=lg2=Vq8l+KCLz3/65A==&vt3=F8dCujumAMEFjB3pdMw=&nk2=F5RFgMLEtD7R5ko=&id2=Vy0Rp2yUPV/cvw==; csg=c80e8a0c; lgc=tb004474346; cancelledSubSites=empty; cookie17=Vy0Rp2yUPV/cvw==; dnk=tb004474346; skt=2667ba283dd77bc8; existShop=MTYzNTUyNDk1MQ==; uc4=id4=0@VXqfvmVX6SyIFcO0kX/2Y6AIwD5E&nk4=0@FY4O6YW3L93fnB8RCIHguzzV7EPmeA==; _cc_=W5iHLLyFfA==; _l_g_=Ug==; sg=656; _nk_=tb004474346; cookie1=VyTxQUOX1QQJnh+mpdj1xrROuwUYVZOt0aeQLznOknk=; thw=cn; hng=CN|zh-CN|CNY|156; v=0; mt=ci=15_1; uc1=existShop=false&cookie15=VT5L2FSpMGV7TQ==&cookie21=U+GCWk/7pY/F&cookie14=Uoe3crgRn7ShNA==&cookie16=VT5L2FSpNgq6fDudInPRgavC+Q==&pas=0; isg=BMjIp0DyPZeJY22l7WH6CEz8mTbacSx7LRbkD4J5FMM2XWjHKoH8C15f1TUt9uRT; l=eBLViLDgqeAA6FWBBOfanurza77OSIRYYuPzaNbMiOCP9TCB5x9dW6ULscY6C3GVh6XXR3yFdUcDBeYBqQAonxv92j-la_kmn; tfstk=cI8fBNDRx-2XIyoauIGzQQ0RfBQOwck5Zm6vGh_59TYPx910VyWI3XxgEN5ON"}
        
    r = requests.get(url,headers=headers)
    r.encoding = r.apparent_encoding
    return r.text
    
    pass
    
def parsePage(ilt,html):
    """解析页面信息html，将商品价格和名称保存到列表ilt中；
    ilt是一个二维列表，其中的每个元素是商品价格和名称的一个一维列表"""
    name = re.findall("(?<=raw_title\":\").*?(?=\",)",html)
    price = re.findall("(?<=view_price\":\").*?(?=\",)",html)
    for i in range(len(name)):
        ilt.append([name[i], price[i]])
    return ilt
    pass
    
def savePriceList(ilt,filename):
    """保存爬取的信息到文件filename中；
    文件是excel类型"""
    ws = openpyxl.Workbook()
    wb = ws.active
    wb.title = '商品信息'
    wb['A1'] = '名称' 
    wb['B1'] = '价格'
    for row in ilt:
            wb.append(row)
    ws.save(filename)
    pass


def main():
    goods =  '扫地机器人'# 要爬取的商品名称
    depth =  2 # 爬取深度，也就是向下爬取的页面数
    start_url = "https://s.taobao.com/search?q=" + goods  # 淘宝搜索页面的url
    infoList = []
    filename = "goods.xlsx"  # 保存信息的文件名
    for i in range(depth):
        try:
            url = start_url + "&s=" + str(44*i)
            html = getHTMLText(url)
            parsePage(infoList,html)
        except:
            continue
    savePriceList(infoList,filename)

main()



